In [4]:
#######  1. 模型准备 #######
# LLMs模型加载
from qwen2_llm import Qwen
llm = Qwen()
llm.invoke('你是谁')

/mnt/workspace/lonhai/LangChain_Deploy/my_langChain_venv/my_venv_first/lib/python3.11/site-packages/_distutils_hack/__init__.py:53: UserWarning: Reliance on distutils from stdlib is deprecated. Users must rely on setuptools to provide the distutils module. Avoid importing distutils or import setuptools first, and avoid setting SETUPTOOLS_USE_DISTUTILS=stdlib. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(


'我是你的AI助手，专注于帮助你解答问题、提供支持和提供建议。有什么可以帮助你的吗？'

In [ ]:
## embedding模型加载
from langchain_huggingface import HuggingFaceEmbeddings

model_name = "../Qwen/Qwen3-Embedding-0.6B"
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': False}
# 加载词嵌入模型
hf = HuggingFaceEmbeddings(
    model_name=model_name, # 模型名称
    model_kwargs=model_kwargs, # 模型参数
    encode_kwargs=encode_kwargs # 模型转向量的参数
)
hf.embed_query('你是')

In [13]:
#######  2. 数据准备：文档读取和切分 #######
# 文档读取
from langchain_community.document_loaders import PyPDFLoader,Docx2txtLoader
# loader = Docx2txtLoader('./data/人事面试常见问题.docx') # 定义读取方法和文档
loader = PyPDFLoader('./data/人事面试常见问题.pdf') # 定义读取方法和文档
# document = loader.load() # 进行读取文档
# document[0].page_content # 文档内容
# document[0].metadata # 文档来源

In [14]:
# 文档切分
from langchain_text_splitters.character import CharacterTextSplitter,RecursiveCharacterTextSplitter
# 定义切分方法
spliter = RecursiveCharacterTextSplitter(
                      chunk_size=200,    # 文档块大小（最大）
                      chunk_overlap=30) # 重叠块大小
# 文档分块
chunk_document = spliter.split_documents(document)

In [16]:
#######  3. 向量存储和检索 #######
from langchain_community.vectorstores import FAISS,Qdrant
# 向量存储
vectorstore = FAISS.from_documents(
    documents=chunk_document, # 待存储的文档对象
    embedding=hf, # 词嵌入模型
)
# 测试，相似性文档搜索
query = '面试时哪些行为可以加分？'
docs = vectorstore.similarity_search(query, k=4)

In [ ]:
#######  4. 检索器 #######
#定义检索器
retriever = vectorstore.as_retriever(
    search_type='similarity', # similarity mmr similarity_score_threshold
    search_kwargs={'k':4,} # 其他搜索参数
)
retriever.invoke(query)

In [19]:
#######  5. 构建QA链 #######
# 编写提示模板
template = '''
基于<start> 和 <end>之间的内容回答问题即可，回答用中文。
<start> {context} <end>
问题是：{question}
'''
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template(template)

# 构建链
from langchain.chains import RetrievalQA
chain = RetrievalQA.from_chain_type(
    llm=llm, # 大模型
    retriever=retriever, # 检索器
    chain_type_kwargs={'prompt':prompt}
)
# 测试
chain.invoke('面试时哪些行为可以加分？')

{'query': '面试时哪些行为可以加分？', 'result': '这些行为有助于提升面试表现，'}

In [23]:
#######  6. 界面开发 #######
import random
import gradio as gr

# 生成结果的函数，调用链回复
def get_response(message, history):
    return chain.invoke(message)['result']

demo = gr.ChatInterface(
    fn=get_response, # 执行的函数
    chatbot=gr.Chatbot(height=500), # 聊天窗口
    textbox=gr.Textbox(placeholder="请输入问题", container=False, scale=7), # 文字输入
    title="人事面试百事通",
    description="我是一个精通人事面试的助手，你可以问我相关的面试的问题。",
    theme="soft", # 主题色
    submit_btn = "发送",
    stop_btn = "停止"
    )

/tmp/ipykernel_10680/2120754964.py:11: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot=gr.Chatbot(height=500), # 聊天窗口
/mnt/workspace/lonhai/LangChain_Deploy/my_langChain_venv/my_venv_first/lib/python3.11/site-packages/gradio/chat_interface.py:328: UserWarning: The gr.ChatInterface was not provided with a type, so the type of the gr.Chatbot, 'tuples', will be used.
  warnings.warn(


In [24]:
demo.launch(server_port=8001,  # 端口
            server_name='127.0.0.1', # 地址
            share=True # 网络穿透
            )

* Running on local URL:  http://127.0.0.1:8001

Could not create share link. Missing file: /root/.cache/huggingface/gradio/frpc/frpc_linux_amd64_v0.3. 

Please check your internet connection. This can happen if your antivirus software blocks the download of this file. You can install manually by following these steps: 

1. Download this file: https://cdn-media.huggingface.co/frpc-gradio-0.3/frpc_linux_amd64
2. Rename the downloaded file to: frpc_linux_amd64_v0.3
3. Move the file to this location: /root/.cache/huggingface/gradio/frpc
